Tradução de endereços de rede (NAT)
==================================

Como já vimos nas aulas, NAT (Network Address Translation) é um serviço utilizado para se reaproveitar endereços IPv4, que se tornaram escassos com o crescimento de número de dispositivos na internet.

Os endereços são reaproveitados ao se configurar um único IPv4 público no gateway de uma rede privada, permitindo que seja alcançável na internet pública. Já na parte interna da rede, são reutilizados endereços de IP, que quando tentam enviar um pacote para fora da rede, são interceptados pelo gateway, tem seus campos de IP e porta de origem substituídos pelo IP público do NAT e uma porta disponível do gateway. O pacote modificado é então encaminhado para seu destino final.

Mais detalhes em: https://www.ciscozine.com/nat-and-pat-a-complete-explanation/

Checando a existência de um NAT
-------------------------------

Para identificar se estamos ou não atrás de um NAT, precisamos de um servidor externo que nos auxilie nesta tarefa. O que ele irá fazer é enviar uma resposta à nossa requisição com o endereço de IP de origem que ele recebeu, ou seja, nosso endereço de IP público.

Quando fizer isto, o pacote de responsa enviado será recebido pelo gateway, o endereço IP e porta de destino serão modificados para o IP local de origem da requisição, que conterá o valor do IP público. 

Existe porém, o caso da nossa provedora de internet executar NAT sobre NAT.
$sua\ rede\rightarrow gateway/modem\ do\ ISP\ com\ NAT\rightarrow roteador\ do\ ISP\ com\ NAT$.

Esse conjunto de procedimentos é parte do STUN (Session Traversal Utilities for NAT), definido nas [RFC 5389](https://www.ietf.org/rfc/rfc5389.txt) e [RFC 3489](https://www.ietf.org/rfc/rfc3489.txt).

Existem diversas ferramentas para isso. Usaremos a biblioteca `stun` para python (`pip install pynat`).
Também precisaremos de um servidor remoto rodando a aplicação de servidor do STUN. 

Existe uma lista de servidores públicos que podem ser utilizados em https://github.com/pradt2/always-online-stun.

Usaremos o servidor STUN do Google.

In [6]:
import pynat

local_port = 8080
nat_type, external_ip, external_port, local_ip = pynat.get_ip_info(source_port=local_port,
                                                        stun_host='stun.l.google.com', 
                                                        stun_port=19302,
                                                        include_internal=True
                                                        )

print(f"NAT Type: {nat_type}")
print(f"Local->External IP: {local_ip}->{external_ip}")
print(f"Local->External PORT: {local_port}->{external_port}")

NAT Type: Full-cone NAT
Local->External IP: 192.168.0.114->189.6.16.145
Local->External PORT: 8080->7619


Podemos ver nossos endereços público e privado, e a porta external para a qual o nosso tráfego do socket interno da porta 10000 foi modificado.

Podemos ver que o nosso NAT também é do tipo "Full cone". Olhando na RFC 3489, vemos

```
Full Cone: A full cone NAT is one where all requests from the
      same internal IP address and port are mapped to the same external
      IP address and port.  Furthermore, any external host can send a
      packet to the internal host, by sending a packet to the mapped
      external address.
```

Traduzindo: o nosso tipo de NAT traduz o tráfego de uma mesma porta de um mesmo usuário externo para um endereço ip público e portas fixos. E também, que um dispositivo na internet pode alcançar um dispositivo interno da rede enviando um pacote para a porta pública associada.

Quais são os diferentes tipos de NAT?
-------------------------------------

Olhando na RFC 3489, vemos que estão previstas quatro variantes:
- Full Cone
- Restricted Cone
- Port Restricted Cone
- Symmetric

O seguinte diagrama ajuda a diferenciarmos umas das outras.

![](07_traducao_de_enderecos_de_rede_NAT/nat_types_huawei.png)
Fonte do diagrama e mais informações: https://info.support.huawei.com/info-finder/encyclopedia/en/NAT.html

No Full Cone NAT, como já vimos, basta que o dispositivo local (PC1 no diagrama) abra uma conexão com um servidor remoto (PC2 no diagrama) para que se torne acessível na internet POR QUALQUER DISPOSITIVO (PC3 no diagrama) através da porta pública mapeada pelo NAT.

No Restricted Cone NAT, apenas o IP e porta de origem (PC1 no diagrama) e o IP de destino (PC2 no diagrama) podem se comunicar. Neste caso, um dispositivo qualquer (PC3 no diagrama) só pode se conectar com o dispositivo local (PC1) caso este inicie uma transmissão primeiro. Este tipo de NAT é bastante restritivo caso ambos os dispositivos estejam atrás de NATs.

No Port Restricted Cone NAT, o comportamento é o mesmo do Restricted Cone NAT, exceto que a porta do dispositivo externo (PC2 no diagrama) também é levada em conta. Ou seja, nem mesmo o servidor remoto (PC2 no diagrama) pode abrir um novo socket com um número de porta distinta e enviar pacotes para uma porta já aberta para o dispositivo local (PC1 no diagrama). Assim como no Restricted Cone NAT, um outro dispositivo qualquer (PC3 no diagrama) requer que o dispositivo local faça uma conexão antes de poderem se comunicar por cada uma das portas desejadas.

No Symmetric NAT, o dispositivo local (PC1 no diagrama) ganha uma nova entrada no NAT para cada servidor remoto de destino (PC2 e PC4 no diagrama), sendo mapeado em diferentes portas externas. Assim como nos cones restritos, é necessário que o dispositivo local (PC1) inicie as conexões com os servidores remotos.

Este último tipo de NAT é ainda mais restritivo que os cones restritos, pois inviabiliza hospedagem de serviços pelo dispositivo local (PC1). Exceto quando existem configurações próprias para fixação da porta, zona desmilitarizada (DMZ), ou que sejam utilizadas ferramentas para o estabelecimento de conexões interativas, todas elas soluções de como se transpor um NAT.

Como transpor um NAT?
------------------------------------

Isto pode ser feito automaticamente, e tipicamente é. 

Uma aplicação local, por exemplo, pode se utilizar de requisições UPnP, definido na [RFC 6970](https://www.ietf.org/rfc/rfc6970.txt) para requisitar ao gateway que abra uma porta externa e mapeie seu tráfego para um dispositivo com IP e porta fixas do dispositivo.

Caso o UPnP esteja desabilitado ([ALTAMENTE RECOMENDADO POR QUESTÕES DE SEGURANÇA](https://www.upguard.com/blog/what-is-upnp)), são utilizados uma série de procedimentos alternativos que compõem o ICE (Interactive Connectivity Establishment), definido na [RFC 8445](https://www.ietf.org/rfc/rfc8445.txt).

No pior cenário possível, onde existe um NAT simétrico e não existem outras alternativas, são utilizados servidores TURN (Traversal Using Relays around NAT), definido na [RFC 8656](https://www.ietf.org/rfc/rfc8656.txt).

Os servidores TURN funcionam como um NAT Full Cone remoto, permitindo que o seu endereço IP público e uma porta seja alocado para diversas conexões entre um dispositivo local (PC1), que se conecta com o servidor TURN através do seu NAT simétrico local, e dispositivos remotos (PC2, PC3, PC4).

Este serviço é feito através do tunelamento (VPN) de tráfego, é intensivo em recursos para os servidores, também introduzindo latência ao serviço.

O vídeo abaixo dá a idéia do funcionamento do ICE (1:41-13:28).

In [7]:
%%html
<iframe id="ytplayer" type="text/html" width="640" height="360"
  src="http://www.youtube.com/embed/MEupfrmigbA?autoplay=1&start=104&end=783"
  frameborder="0"
  />